In [1]:
import pandas as pd 
import numpy as np
import datacompy
import pdftotext
import re

In [2]:
PDF_FILE = 'floor_plan_001.pdf'
EXCEL_FILE = 'fp_material_001.xls'

In [3]:
# Loading the PDF into a string variable
with open(PDF_FILE, "rb") as f:
    pdf = pdftotext.PDF(f)
    
# Joining the pdf pages into one variable
full_text = '\n'.join(pdf)

In [4]:
# Regular expression used to extract the data (use https://regex101.com/ to get an in-depth explanation)
reg= r"(\d+(x\d+)?)\s+[A-Z]\d?(\d{2})\s+(\d+)[\s\-]"

pattern = re.compile(reg)
bar_lst = []

for nbars, _, diameter, barmark in re.findall(pattern, full_text):
    # Replacing the 'x' by '*' and evaluate the math operation
    nbars = eval(nbars.replace('x', '*'))
    # Storing the data into a list
    bar_lst.append([nbars, diameter, barmark])
    
# Creating the dataframe from list
df = pd.DataFrame(bar_lst, columns=['nbars', 'diameter', 'barmark']).astype(int)

df1 = df

# Sum grouping by barmark and diameter as key
df1 = df1.groupby(['barmark', 'diameter'], as_index=False)['nbars'].sum()

# Optional: sorting the rows
df1 = df1.sort_values(['barmark', 'diameter', 'nbars'], ascending=True)

# Placing the columnns in order
df1 = df1[['nbars', 'diameter', 'barmark']].astype(int)

df1.head()

,nbars,diameter,barmark
0,140,12,1
1,18,12,2
2,22,12,3
3,60,12,4
4,10,12,5


In [5]:
# Loading the Excel in a dataframe
df2 = pd.read_excel(EXCEL_FILE, sheet_name='Sheet')

# Identifying the corresponding columns for barmark, diameter and nbars (note that this could be also parametrized)
df2 = df2[[df2.columns[2], df2.columns[3], df2.columns[6]]]

# Renaming the columns
df2.columns = ['barmark', 'diameter', 'nbars']

# Dropping NaN rows
df2.dropna(inplace=True)

# Selecting only those rows with numberic barmark.
df2 = df2[df2['barmark'].str.contains('\d+')]

# Deleting the letter of the diameter
df2 = df2.replace('[A-Z]', '', regex=True)

# Optional: sorting the rows
df2 = df2.sort_values(['barmark', 'diameter', 'nbars'], ascending=True)

# Placing the columnns in order
df2 = df2[['nbars', 'diameter', 'barmark']].astype(int)

df2.head()

,nbars,diameter,barmark
8,140,12,1
9,18,12,2
10,22,12,3
11,60,12,4
12,10,12,5


In [6]:
compare = datacompy.Compare(
    df1,
    df2,
    join_columns=['diameter', 'barmark', 'nbars'],  
    df1_name='drawing',
    df2_name='excel')
compare.matches(ignore_extra_columns=True)

print(compare.report())

DataComPy Comparison
--------------------

DataFrame Summary
-----------------

  DataFrame  Columns  Rows
0   Drawing        3    76
1     Excel        3    76

Column Summary
--------------

Number of columns in common: 3
Number of columns in Drawing but not in Excel: 0
Number of columns in Excel but not in Drawing: 0

Row Summary
-----------

Matched on: diameter, barmark, nbars
Any duplicates on match values: No
Absolute Tolerance: 0
Relative Tolerance: 0
Number of rows in common: 75
Number of rows in Drawing but not in Excel: 1
Number of rows in Excel but not in Drawing: 1

Number of rows with some compared columns unequal: 0
Number of rows with all compared columns equal: 75

Column Comparison
-----------------

Number of columns compared with some values unequal: 0
Number of columns compared with all values equal: 3
Total number of values which compare unequal: 0

Sample Rows Only in Drawing (First 10 Columns)
----------------------------------------------

    nbars  diameter  

In [12]:
# Comparing the values between df1, df2 and df for an specific barmark
bm=51

# df1: the aggregate of df by barmark and diameter
df1[df1['barmark'] == bm]

,nbars,diameter,barmark
50,18,12,51


In [13]:
# df2: the excel content
df2[df2['barmark'] == bm]

,nbars,diameter,barmark
87,24,12,51


In [14]:
# df: the drawing content without any aggregation
df[df['barmark'] == bm]

,nbars,diameter,barmark
9,6,12,51
48,6,12,51
81,6,12,51
